In [1]:
import os, config, pickle, nbimporter, random, _1_extract, numpy as np, pandas as pd
Species, Data = _1_extract.Species, _1_extract.Data

Importing Jupyter notebook from _1_extract.ipynb


# Processing the lxmx data

## Goals:
* construct projection matrix $P$ for data
* for each $P$ compute dominant & subdominant eigenvalues and their logs, $r, r_1$
* for each $P$ compute $T_c, T_g, V, r_{0a}, r_{0b}, d, G, K$ (see theory for reference)

In [2]:
SPECIES = config.load_pickle(os.path.join(config.OUTPUT_DIR, 'species.pkl'))

## Auxillary Classes

#### Processing
This class will handle all of the goals mentioned above and augment the Data object created in the data extraction step.

In [79]:
class Processing:
    
    @staticmethod
    def process(species):
        for data in species.data:
            matrix = data.matrix
            ages = data.ages
            l_a = matrix['l(a)'].to_numpy()
            f_a = matrix['f(a)'].to_numpy()
            p_a = matrix['p(a)'].to_numpy()
            data.leslie = Processing.generate_leslie(p_a, f_a)
            eigs = Processing.compute_eig(data.leslie)
            data.eigen = {'vals': eigs[0], 'right': eigs[1], 'left': eigs[2], 'r_i': eigs[3]}
            data.eigen['damping'] = np.exp(data.eigen['r_i'][1] - data.eigen['r_i'][0])
            data.derivatives = Processing.compute_derivatives(l_a, f_a, ages, data.eigen)
            data.approximations = Processing.compute_approximations(data.derivatives, data.eigen)

            
    @staticmethod
    def generate_leslie(p_a, f_a):
        N = p_a.shape[0]
        leslie = np.zeros((N, N))
        leslie[0] = f_a
        np.fill_diagonal(leslie[1:, :-1], p_a)
        return leslie

    @staticmethod
    def compute_eig(leslie):
        #RIGHT
        r_vals, r_vecs = np.linalg.eig(leslie)
        ix_r = r_vals.argsort()
        ix_r = ix_r[::-1]
        r_vals = r_vals[ix_r]
        r_vecs = r_vecs[ix_r]
        #LEFT
        l_vals, l_vecs = np.linalg.eig(leslie.T)
        ix_l = l_vals.argsort()
        ix_l = ix_l[::-1]
        l_vals = l_vals[ix_l]
        l_vecs = l_vecs[ix_l]
        assert np.linalg.norm(l_vals - r_vals) < .00001 #check eigenvalues are the same
        r_eigs = r_vecs[:2]
        l_eigs = l_vecs[:2]
        vals = r_vals[:2]
        r_i = np.log(np.real(vals))
        return vals, r_eigs, l_eigs, r_i 
    
    def compute_approximations(deriv, eig):
        approx = {}
        approx['d'] = np.exp(-2*np.pi*deriv['V']/deriv['T_c']**3)
        approx['r_0a'] = r_0a = np.log(deriv['R_0'])/deriv['T_c'] #without dispersion
        approx['r_0b'] = r_0a + deriv['V']*np.log(deriv['R_0'])**2/(deriv['T_c']**3)  #with dispersion
        approx['r_1a'] = eig['r_i'][0] - 2 * np.pi / (deriv['T_c']**3)
        approx['s_1a'] =  2 * np.pi / deriv['T_c'] - np.pi * deriv['V'] * np.log(deriv['R_0'])/ (deriv['T_c']**3)
        return approx
    
    @staticmethod
    def compute_derivatives(l_a, f_a, ages, eigen):
        derivatives = {}
        derivatives['R_0'] = R_0 = np.sum(l_a * f_a)
        derivatives['T_c'] = T_c =  1 / R_0 * np.sum(ages * l_a * f_a)
        derivatives['V'] = V = 1 / R_0 * np.sum((ages - T_c)**2 * l_a * f_a)
        derivatives['G'] = G = 1 / (R_0 * V**1.5) * np.sum((ages - T_c)**3 * l_a * f_a)
        derivatives['K'] = K =  1 / (R_0 * V**2) * np.sum((ages - T_c)**4 * l_a * f_a)
        return derivatives
    
    @staticmethod
    def check_derivatives(species):
        for data in species.data:
            T_c_reported = data.reported['T_c'][1]
            R_0_reported = data.reported['R_0']
            V_reported = data.reported['V']
            exceptions = []
            if not (abs(data.derivatives['T_c'] - T_c_reported) < .01):
                exceptions.append('T_c calculated: {}, reported: {}'.format(data.derivatives['T_c'], T_c_reported))
            if not (R_0_reported is None or abs(data.derivatives['R_0'] - R_0_reported) < .01): 
                exceptions.append('R_0 calculated: {}, reported: {}'.format(data.derivatives['R_0'], R_0_reported))
            if not (V_reported is None or abs(data.derivatives['V'] - V_reported) < .01): 
                exceptions.append('V calculated: {}, reported: {}'.format(data.derivatives['V'], V_reported))
            if len(exceptions) > 0:
                combined = '; '
                combined = combined.join(exceptions)
                raise Exception('For {}, {}'.format(species.name, combined))
                

In [80]:
for s in SPECIES:
    Processing.process(SPECIES[s])
    try:
        Processing.check_derivatives(SPECIES[s])
    except Exception as e:
        print('**********')
        print(e)


**********
For Spermophilus lateralis, T_c calculated: 3.3807044887780555, reported: 3.511; R_0 calculated: 0.9623999999999999, reported: 0.998; V calculated: 2.2432498777603995, reported: 2.618
**********
For Lepus europaeus, T_c calculated: 13.279863097655307, reported: 1.217; V calculated: 13.720307623123851, reported: 0.264
**********
For Capreolus capreolus, T_c calculated: 4.555876774257954, reported: 4.754; V calculated: 3.9787695061941166, reported: 4.814
**********
For Sylvilagus floridanus, T_c calculated: 9.183254625265562, reported: 0.763; V calculated: 36.19810073077049, reported: 0.253
**********
For Pteropus conspicillatus, T_c calculated: 4.001943479485495, reported: 4.76; V calculated: 4.921872103086531, reported: 3.302
**********
For Lynx rufus, T_c calculated: 2.169733372399648, reported: 2.67
**********
For Arctocephalus australis, T_c calculated: 6.323251800587961, reported: 5.823
**********
For Liomys adspersus, T_c calculated: 9.254684302074919, reported: 0.771; 

/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in log
/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


## Working with the Species objects

You can acccess the calculated derivatives as follows:

In [81]:
s = random.choice(list(SPECIES.keys()))

In [82]:
SPECIES[s]

Kobus ellipsiprymnus: 1 data entries, notes: []

In [83]:
SPECIES[s].data[0].eigen.keys()

dict_keys(['vals', 'right', 'left', 'r_i', 'damping'])

In [84]:
SPECIES[s].data[0].eigen['vals']

array([0.86348566+0.j        , 0.54523508+0.44448616j])

In [85]:
SPECIES[s].data[0].eigen['r_i']

array([-0.14677799, -0.60653823])

In [86]:
SPECIES[s].data[0].eigen['damping']

0.6314350159201683

In [87]:
SPECIES[s].data[0].derivatives

{'R_0': 0.37936000000000003,
 'T_c': 5.19627794179671,
 'V': 5.2802750539166405,
 'G': 0.5214141023739632,
 'K': 2.3734262060649387}

In [88]:
SPECIES[s].data[0].approximations

{'d': 0.789416766164705,
 'r_0a': -0.18653152651220548,
 'r_0b': -0.15117507577789258,
 'r_1a': -0.191559911936404,
 's_1a': 1.3237675437049226}

In [89]:
SPECIES[s].data[0].reported

{'T_c': [None, 5.196], 'V': 5.28, 'R_0': None}

In [78]:
save = True
if save:
    config.save_pickle(SPECIES, os.path.join(config.OUTPUT_DIR, 'species_processed.pkl'))

## Bugs

In [77]:
Processing.process(SPECIES['Ursus arctos horribilis'])

/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in log
/home/kaan/anaconda3/envs/bio172_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [76]:
Processing.check_derivatives(SPECIES['Lepus europaeus'])

Exception: For Lepus europaeus, T_c calculated: 13.279863097655307, reported: 1.217; V calculated: 13.720307623123851, reported: 0.264